In [ ]:
!pip install -r https://raw.githubusercontent.com/GoogleCloudPlatform/python-docs-samples/main/people-and-planet-ai/land-cover-classification/requirements.txt

In [ ]:
#@title My Google Cloud resources
project = "" #@param {type:"string"}
bucket = "" #@param {type:"string"}
location = "us-central1" #@param {type:"string"}

# Load from environment variables if available
import os
project = os.environ.get('GOOGLE_CLOUD_PROJECT', project)
bucket = os.environ.get('CLOUD_STORAGE_BUCKET', bucket)
location = os.environ.get('CLOUD_LOCATION', location)

# Quick input validations
assert project, '⚠️ Please provide a Google Cloud project ID'
assert bucket, '⚠️ Please provide a Cloud Storage bucket name'
assert not bucket.startswith('gs://'), f'⚠️ Please remove the gs:// prefix from the bucket name: {bucket}'
assert location, '⚠️ Please provide a Google Cloud location'

# Workaround: Use legacy authentication method (b/229402252).
os.environ['USE_AUTH_EPHEM'] = ''

# Authenticate
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
import google.auth

credentials, _ = google.auth.default(
    scopes=[
        'https://www.googleapis.com/auth/cloud-platform',
        'https://www.googleapis.com/auth/earthengine',
    ]
)
ee.Initialize(credentials, project=project)

In [ ]:
CLASSIFICATIONS = {
    '💧 Water':              '449cda',
    '🌳 Trees':              '3d7c49',
    '🌾 Grass':              '8ab052',
    '🌿 Flooded vegetation': '7c86c7',
    '🚜 Crops':              'e19832',
    '🪴 Shrub and scrub':    'dfc35b',
    '🏗️ Built-up areas':     'c52918',
    '🪨 Bare ground':        'a29d90',
    '❄️ Snow and ice':       'b4a0e0',
}
PALETTE = list(CLASSIFICATIONS.values())

def display_legend():
  reset_color = '\u001b[0m'
  colored = lambda red, green, blue: f"\033[48;2;{red};{green};{blue}m"
  for name, color in CLASSIFICATIONS.items():
    red   = int(color[0:2], 16)
    green = int(color[2:4], 16)
    blue  = int(color[4:6], 16)
    print(f"{colored(red, green, blue)}   {reset_color} {name}")

display_legend()

In [ ]:
import folium

def get_sentinel2_image(start_date: str, end_date: str) -> ee.Image:
    def mask_sentinel2_clouds(image: ee.Image) -> ee.Image:
        CLOUD_BIT = 10
        CIRRUS_CLOUD_BIT = 11
        bit_mask = (1 << CLOUD_BIT) | (1 << CIRRUS_CLOUD_BIT)
        mask = image.select("QA60").bitwiseAnd(bit_mask).eq(0)
        return image.updateMask(mask)

    return (
        ee.ImageCollection("COPERNICUS/S2")
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .map(mask_sentinel2_clouds)
        .median()
    )

sentinel2_image = get_sentinel2_image('2020-1-1', '2021-01-1')

vis_params = {
  'min': 0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}
folium.Map(
    location=[38.6561723, -9.0624997],
    zoom_start=12,
    tiles=sentinel2_image.getMapId(vis_params)['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)

In [ ]:
import folium

def get_landcover_image() -> ee.Image:
    # Remap the ESA classifications into the Dynamic World classifications
    fromValues = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
    toValues = [1, 5, 2, 4, 6, 7, 8, 0, 3, 3, 7]
    return (
        ee.ImageCollection("ESA/WorldCover/v100")
        .first()
        .select("Map")
        .remap(fromValues, toValues)
        .rename("landcover")
    )

landcover_image = get_landcover_image()

vis_params = {
  'bands': ['landcover'],
  'palette': PALETTE,
}
folium.Map(
    location=[38.6561723, -9.0624997],
    zoom_start=12,
    tiles=landcover_image.getMapId(vis_params)['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)

In [ ]:
display_legend()

In [ ]:
import csv

# Get the first row from the CSV to examine it.
with open('data/points-small.csv') as f:
  row = next(csv.DictReader(f))
  coords = (float(row["lon"]), float(row["lat"]))
  print(f"coords as (lon, lat): {coords}")

In [ ]:
from typing import Tuple, List
import io
import numpy as np
import requests

INPUT_BANDS = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]
OUTPUT_BANDS = ["landcover"]

def get_patch(coords: Tuple[float, float], bands: List[str], patch_size: int, scale: int) -> np.ndarray:
    sentinel2_image = get_sentinel2_image("2020-1-1", "2021-1-1")
    landcover_image = get_landcover_image()
    image = sentinel2_image.addBands(landcover_image)

    point = ee.Geometry.Point(coords)
    url = image.getDownloadURL(
        {
            "region": point.buffer(scale * patch_size / 2, 1).bounds(1),
            "dimensions": [patch_size, patch_size],
            "format": "NPY",
            "bands": bands,
        }
    )
    np_bytes = requests.get(url, stream=True).content
    return np.load(io.BytesIO(np_bytes), allow_pickle=True)

sentinel2_image = get_sentinel2_image("2020-1-1", "2021-1-1")
landcover_image = get_landcover_image()
image = sentinel2_image.addBands(landcover_image)

patch = get_patch(coords, INPUT_BANDS + OUTPUT_BANDS, patch_size=8, scale=10)
print(f"patch shape={patch.shape} bands={len(patch.dtype)}")
print(f"dtype: {patch.dtype}")

In [ ]:
import tensorflow as tf

def serialize(patch: np.ndarray) -> bytes:
  features = {
      name: tf.train.Feature(
          float_list=tf.train.FloatList(value=patch[name].flatten())
      )
      for name in INPUT_BANDS + OUTPUT_BANDS
  }
  example = tf.train.Example(features=tf.train.Features(feature=features))
  return example.SerializeToString()

serialized = serialize(patch)
print(f"serialized: {type(serialized).__name__}[{len(serialized)}]")

In [ ]:
training_patch_size = 8 #@param {type:"integer"}

# Load from environment variables if available.
training_patch_size = int(os.environ.get('TRAINING_PATCH_SIZE', training_patch_size))

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
import random

training_validation_ratio = [90, 10]

def split_dataset(element, num_partitions) -> int:
  return random.choices([0, 1], weights=training_validation_ratio)[0]

with open("data/points-small.csv") as f:
  points = [(float(row["lon"]), float(row["lat"])) for row in csv.DictReader(f)]

beam_options = PipelineOptions([], direct_num_workers=0)
with beam.Pipeline(options=beam_options) as pipeline:
  training_data, validation_data = (
      pipeline
      | "Create points" >> beam.Create(points)
      | "Get patch" >> beam.Map(get_patch, INPUT_BANDS + OUTPUT_BANDS, training_patch_size, scale=10)
      | "Serialize" >> beam.Map(serialize)
      | "Split dataset" >> beam.Partition(split_dataset, 2)
  )

  training_data | "Write training data" >> beam.io.tfrecordio.WriteToTFRecord(
      "training-data", file_name_suffix=".tfrecord.gz"
  )
  validation_data | "Write validation data" >> beam.io.tfrecordio.WriteToTFRecord(
      "validation-data", file_name_suffix=".tfrecord.gz"
  )

In [ ]:
import tensorflow as tf

filenames = tf.data.Dataset.list_files(f"training-data*.tfrecord.gz")
dataset = tf.data.TFRecordDataset(filenames, compression_type='GZIP')
for x in dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(x.numpy())
  for _, feature in example.ListFields():
    for _, field in feature.ListFields():
      for name, values in sorted(field.items()):
        print(f"{name}: float[{len(values.float_list.value)}]")

In [ ]:
print(f"Number of classifications: {len(CLASSIFICATIONS)}")
print(f"Training patch size: {training_patch_size}")
print(f"Training patch area: {training_patch_size * training_patch_size}")

In [ ]:
from typing import Dict, Tuple

def preprocess(patch: Dict[str, tf.Tensor]) -> Tuple[tf.Tensor, tf.Tensor]:
    inputs = tf.stack([patch[band] for band in INPUT_BANDS], axis=-1)
    outputs = tf.one_hot(tf.cast(patch["landcover"], tf.uint8), len(CLASSIFICATIONS))
    return (inputs, outputs)

input_shape = (training_patch_size, training_patch_size)
features_dict = {
    band_name: tf.io.FixedLenFeature(input_shape, tf.float32)
    for band_name in INPUT_BANDS + OUTPUT_BANDS
}

def read_dataset(file_pattern: str) -> tf.data.Dataset:
  return (
      tf.data.Dataset.list_files(file_pattern)
      .interleave(lambda filename: tf.data.TFRecordDataset(filename, compression_type="GZIP"))
      .batch(4)
      .map(lambda batch: tf.io.parse_example(batch, features_dict))
      .map(preprocess)
  )

training_dataset = read_dataset(f"training-data*.tfrecord.gz")
validation_dataset = read_dataset(f"validation-data*.tfrecord.gz")
for inputs, outputs in training_dataset.take(1):
  print(f"inputs:  {inputs.shape}")
  print(f"outputs: {outputs.shape}")

In [ ]:
kernel_size = 5 #@param {type:"integer"}

# Load from environment variables if available.
kernel_size = int(os.environ.get('KERNEL_SIZE', kernel_size))

In [ ]:
import tensorflow as tf

normalization = tf.keras.layers.Normalization(name="Normalized")
normalization.adapt(training_dataset.map(lambda x, _: x))

model = tf.keras.Sequential([
    tf.keras.Input(shape=(None, None, len(INPUT_BANDS)), name="Inputs"),
    normalization,
    tf.keras.layers.Conv2D(filters=32, kernel_size=kernel_size, activation="relu", name="Conv2D"),
    tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=kernel_size, activation="relu", name="DeConv2D"),
    tf.keras.layers.Dense(len(CLASSIFICATIONS), activation="softmax", name="LandCover"),
])
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
    
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.fit(
    training_dataset.shuffle(10),
    validation_data=validation_dataset,
    epochs=10,
)
model.save('local-model')

In [ ]:
prediction_patch_size = 64 #@param {type:"integer"}

# Load from environment variables if available.
prediction_patch_size = int(os.environ.get('PREDICTION_PATCH_SIZE', prediction_patch_size))

In [ ]:
from typing import Dict

import apache_beam as beam
from apache_beam.io.filesystems import FileSystems
from apache_beam.options.pipeline_options import PipelineOptions
import ee

# The fire was in November 2018.
years = [2017, 2018, 2019, 2020, 2021]

regions = [
    ('🔥 Camp fire', {'lat': 39.7818, 'lon': -121.5124, 'year': year})
    for year in years
]

def fetch_input_patch(name: str, region: Dict[str, float], patch_size: int = 256, scale: int = 10) -> None:
  image = get_sentinel2_image(f"{region['year']}-01-01", f"{region['year']}-12-31")
  point = ee.Geometry.Point([region['lon'], region['lat']])
  url = image.getDownloadURL(
      {
          "region": point.buffer(scale * patch_size / 2, 1).bounds(1),
          "dimensions": [patch_size, patch_size],
          "format": "NPY",
          "bands": INPUT_BANDS,
      }
  )
  np_bytes = requests.get(url).content
  structured_array = np.load(io.BytesIO(np_bytes), allow_pickle=True)
  patch = np.stack([structured_array[band] for band in INPUT_BANDS], axis=-1)

  filename = f"predictions/{name}/{region['year']}.npz"
  with FileSystems.create(filename) as f:
    np.savez_compressed(f, patch=patch)
    print(f"Created {filename}")

beam_options = PipelineOptions([], direct_num_workers=0)
with beam.Pipeline(options=beam_options) as pipeline:
  (
      pipeline
      | "Create regions" >> beam.Create(regions)
      | "Fetch patch" >> beam.MapTuple(fetch_input_patch, prediction_patch_size, scale=10)
  )

In [ ]:
!ls -lh "predictions/🔥 Camp fire"

In [ ]:
from apache_beam.io.filesystems import FileSystems
import numpy as np

def load_patch(filename: str) -> np.ndarray:
  with FileSystems.open(filename) as f:
    return np.load(f)['patch']

input_batch = np.stack([
    load_patch(f"predictions/🔥 Camp fire/{year}.npz")
    for year in years
])
print(f"input_batch: {input_batch.dtype} {input_batch.shape}")

In [ ]:
import matplotlib.pyplot as plt

def render_rgb_images(values: np.ndarray, min=0.0, max=1.0, gamma=1.0) -> np.ndarray:
  scaled_values = (values - min) / (max - min)
  gamma_corrected_values = scaled_values ** (1.0 / gamma)
  rgb_values = gamma_corrected_values * 255
  return rgb_values.astype(np.uint8)

def render_sentinel2(input_batch: np.ndarray) -> np.ndarray:
  red   = input_batch[..., 3]  # B4
  green = input_batch[..., 2]  # B3
  blue  = input_batch[..., 1]  # B2
  rgb_input_batch = np.stack([red, green, blue], axis=-1)
  return render_rgb_images(rgb_input_batch, max=3000)

sentinel2_images = render_sentinel2(input_batch)
print(f"sentinel2 images: {sentinel2_images.dtype} {sentinel2_images.shape}")

# Show one of the images to see how the Landsat input images look like.
plt.imshow(sentinel2_images[0])

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(f"local-model")
model.summary()

In [ ]:
probabilities = model.predict(input_batch)
print(f"probabilities: {probabilities.dtype} {probabilities.shape}")

In [ ]:
def render_classifications(probabilities: np.ndarray, palette: list) -> np.ndarray:
  # Create a color map from a hex color palette.
  xs = np.linspace(0, len(palette) - 1, 256)
  indices = np.arange(len(palette))
  color_map = np.array([
        np.interp(xs, indices, [int(c[0:2], 16) for c in palette]),  # reds
        np.interp(xs, indices, [int(c[2:4], 16) for c in palette]),  # greens
        np.interp(xs, indices, [int(c[4:6], 16) for c in palette]),  # blues
  ]).astype(np.uint8).transpose()

  # Convert the probabilities into prediction indices and map them to the color map.
  predictions = np.argmax(probabilities, axis=-1)
  color_indices = (predictions / len(CLASSIFICATIONS) * 255).astype(np.uint8)
  return np.take(color_map, color_indices, axis=0)

def render_landcover(probabilities: np.ndarray) -> np.ndarray:
  palette = list(CLASSIFICATIONS.values())
  return render_classifications(probabilities, palette)

# Render all our model probability predictions into RGB images.
landcover_images = render_landcover(probabilities)
print(f"landcover images: {landcover_images.dtype} {landcover_images.shape}")

# Show one of the images to see how the land cover predictions look like.
plt.imshow(landcover_images[0])

display_legend()

In [ ]:
def display_land_cover_change(sentinel2_images: np.ndarray, landcover_images: np.ndarray):
  img_size = 3
  figsize = (img_size * len(years), img_size * 2)
  figure, plot_rows = plt.subplots(2, len(years), figsize=figsize, sharex=True, sharey=True)
  figure.tight_layout(pad=0)

  for title, plot, image in zip(years, plot_rows[0], sentinel2_images):
    plot.set_title(title if title != 2018 else '2018 (fire)')
    plot.imshow(image)
  for plot, image in zip(plot_rows[1], landcover_images):
    plot.imshow(image)

  display_legend()

display_land_cover_change(sentinel2_images, landcover_images)

In [ ]:
import os
import imageio
from IPython.display import Image

def land_cover_change_gif(sentinel2_images: np.ndarray, landcover_images: np.ndarray):
  display_legend()

  frames = np.concatenate([sentinel2_images, landcover_images], axis=2)
  imageio.mimwrite('landcover-change.gif', frames, duration=1)
  return Image(open('landcover-change.gif', 'rb').read())

land_cover_change_gif(sentinel2_images, landcover_images)

In [ ]:
import subprocess

cmd = [
    f"python",
    f"create_datasets.py",
    f"--training-file=gs://{bucket}/land-cover/training-data",
    f"--validation-file=gs://{bucket}/land-cover/validation-data",
    f"--patch-size={training_patch_size}",
    f"--points-file=data/points.csv",
    f"--runner=DataflowRunner",
    f"--project={project}",
    f"--region={location}",
    f"--temp_location=gs://{bucket}/land-cover/temp",
    f"--setup_file=./setup.py",
]
subprocess.run(cmd, check=True)